<a href="https://colab.research.google.com/github/Aggraj/Deep-Learning-CS-6910/blob/main/FFNNv6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np

class Feedforwardneuralnetwork:

    def __init__(self,n_inputs,n_hidden,n_outputs):

        self.n_inputs   = n_inputs
        self.n_outputs  = n_outputs
        self.n_hidden   = n_hidden
        self.weights    = []
        self.biases     = []

        layers = [self.n_inputs] + self.n_hidden + [self.n_outputs]
        for i in range(len(n_hidden)+1):
            self.weights.append(np.random.randn(layers[i+1],layers[i]))
            self.biases.append(np.random.randn(layers[i+1],1))



    def sigmoid(self,x):
        return 1 / ( 1 + np.exp(-x))

    def softmax(self,x):
        soft = np.zeros(x.shape)
        for i in range(0, x.shape[1]):
            numr = np.exp(x[:, i])
            soft[:, i] = numr/np.sum(numr)
        return soft

    def forward_propagation(self,input):

        self.intermidiate_inputs = []
        self.post_outputs  = []
        W      = self.weights
        b      = self.biases

        k=0
        self.intermidiate_inputs.append(np.matmul(W[k],input)+b[k])
        self.post_outputs.append(self.sigmoid(self.intermidiate_inputs[k]))

        for k in range(1,len(self.n_hidden)):
            self.intermidiate_inputs.append(np.matmul(W[k],self.post_outputs[k-1])+b[k])
            self.post_outputs.append(self.sigmoid(self.intermidiate_inputs[k]))

        k=len(self.n_hidden)
        self.intermidiate_inputs.append(np.matmul(W[k],self.post_outputs[k-1])+b[k])
        self.post_outputs.append(self.softmax(self.intermidiate_inputs[k]))

        return self.post_outputs[-1]

    def back_propagation(self,train_images,train_labels):

        g_weights = [0]*(len(self.weights))
        g_biases  = [0]*(len(self.biases))
        g_a       = [0]*(len(self.n_hidden)+1)
        g_h       = [0]*(len(self.n_hidden)+1)
        n_samples = train_images.shape[0]  # Change depending on the dimensions of data


        for k in reversed(range(len(self.n_hidden)+1)):
            if k == len(self.n_hidden):
                g_a[k] = self.post_outputs[k] - train_labels  # keep or remove T depending on the dimensions of data
                #g_weights[k] = (1/n_samples)*np.matmul(g_a[k],self.post_outputs[k-1].T)
                #g_biases[k]  = (1/n_samples)*np.sum(g_a[k], axis=1, keepdims = True)
            else:
                g_h[k] = (1/n_samples)*np.matmul(self.weights[k+1].T,g_a[k+1])
                g_a[k] = (1/n_samples)*np.multiply(g_h[k],np.multiply(self.sigmoid(self.intermidiate_inputs[k]),(1-self.sigmoid(self.intermidiate_inputs[k]))))

            if k == 0:
                g_weights[k] = (1/n_samples)*np.matmul(g_a[k],train_images.T)
            else:
                g_weights[k] = (1/n_samples)*np.matmul(g_a[k],self.post_outputs[k-1].T)

            g_biases[k]  = (1/n_samples)*np.sum(g_a[k], axis=1, keepdims = True)
        return g_weights,g_biases



    def train_model(self,train_images,train_labels,epochs,learning_rate,opt='gd',gamma = 0.9,beta = 0.99): 
         
        pre_delta_w = np.multiply(self.weights,0)
        pre_delta_b = np.multiply(self.biases,0)
        vw = 0
        vb = 0
        eps = 1e-8
        lr_w = 0
        lr_b = 0



        for i in range(epochs+1):
            output =  self.forward_propagation(train_images)
            g_weights,g_biases = self.back_propagation(train_images,train_labels)
            if opt == 'gd':
                delta_w = np.multiply(learning_rate,g_weights)
                delta_b = np.multiply(learning_rate,g_biases)
          
            if opt == 'mgd':
                 delta_w = np.multiply(gamma,pre_delta_w) + np.multiply(learning_rate,g_weights)
                 delta_b = np.multiply(gamma,pre_delta_b) + np.multiply(learning_rate,g_biases)
                 pre_delta_w = delta_w
                 pre_delta_b = delta_b

            if opt == 'ngd':

                self.weights = self.weights - np.multiply(gamma,pre_delta_w)
                self.biases  = self.biases - np.multiply(gamma,pre_delta_b)
                output =  self.forward_propagation(train_images)
                g_weights,g_biases = self.back_propagation(train_images,train_labels)

                delta_w = np.multiply(gamma,pre_delta_w) + np.multiply(learning_rate,g_weights)
                delta_b = np.multiply(gamma,pre_delta_b) + np.multiply(learning_rate,g_biases)
                
                pre_delta_w = delta_w
                pre_delta_b = delta_b
                
            if opt == 'Rmsprop': 
                 
                vw = np.multiply(vw,beta) + np.multiply(1-beta,np.power(g_weights,2))
                vb = np.multiply(vb,beta) + np.multiply(1-beta,np.power(g_biases,2))
                lr_w = learning_rate/np.power(vw+eps,1/2)
                lr_b = learning_rate/np.power(vb+eps,1/2)
          
                delta_w = np.multiply(g_weights,lr_w)
                delta_b = np.multiply(g_biases,lr_b)

            self.weights = self.weights - delta_w
            self.biases  = self.biases  - delta_b
            train_loss = -np.sum(np.multiply(train_labels,np.log(output)))/train_labels.shape[1]    
            #print('training_loss for epoch {} = {}'.format(i,train_loss))

            Validate = Model1.forward_propagation(train_val_images.T)
            out_class=(np.argmax(Validate,axis=0))
            target_class_validate=(np.argmax(train_val_labels,axis=1))
            acc2 = 100*np.sum(out_class==target_class_validate)/Validate.shape[1]
            print('Epoch {}: training_loss = {}, Validation accuracy = {}'.format(i,train_loss,acc2))


        return train_loss

In [ ]:
import tensorflow as tf
from sklearn.metrics import log_loss
from keras.datasets import fashion_mnist
output_classes = 10
Model1  = Feedforwardneuralnetwork(28*28,[64,64],output_classes)
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
n_samples = train_images.shape[0]
train_images = train_images.reshape(n_samples,-1)
train_imag = train_images[:50000,:]
train_val_images = train_images[50000:,:]
labels = np.zeros((train_labels.shape[0],output_classes))
for i in range(train_labels.shape[0]):
  e = [0.0]*output_classes
  e[train_labels[i]] = 1.0
  labels[i] = e
train_label = labels[:50000,:]
train_val_labels = labels[50000:,:]
mean = train_imag.mean(axis=0)
std  = train_imag.std(axis = 0)
train_imag = (train_imag - mean)/std
train_val_images = (train_val_images - mean)/std
epochs = 100
learning_rate = 0.01
(tr_loss) = Model1.train_model(train_imag.T,train_label.T,epochs,learning_rate,'Rmsprop')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:81: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:119: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/loca

Epoch 0: training_loss = 9.19240949452164, Validation accuracy = 6.51
Epoch 1: training_loss = 5.667882941775941, Validation accuracy = 7.92
Epoch 2: training_loss = 4.7216461303373505, Validation accuracy = 8.78
Epoch 3: training_loss = 4.154335473588553, Validation accuracy = 12.29
Epoch 4: training_loss = 3.8013016054403654, Validation accuracy = 14.26
Epoch 5: training_loss = 3.5864691005891403, Validation accuracy = 16.21
Epoch 6: training_loss = 3.413110765200276, Validation accuracy = 17.57
Epoch 7: training_loss = 3.266358178711345, Validation accuracy = 19.39
Epoch 8: training_loss = 3.138507156298816, Validation accuracy = 20.59
Epoch 9: training_loss = 3.025308246887121, Validation accuracy = 21.88
Epoch 10: training_loss = 2.9238762308663953, Validation accuracy = 22.87
Epoch 11: training_loss = 2.832195353402546, Validation accuracy = 24.15
Epoch 12: training_loss = 2.7487192984972295, Validation accuracy = 25.11
Epoch 13: training_loss = 2.672293715982815, Validation accu

In [ ]:

output = Model1.forward_propagation(train_imag.T)
out_class=(np.argmax(output,axis=0))
target_class=(np.argmax(train_label,axis=1))
acc2 = 100*np.sum(out_class==target_class)/output.shape[1]
print(acc2)

69.118


In [ ]:
(tr_loss) = Model1.train_model(train_imag.T,train_label.T,10,learning_rate/50,'ngd')

In [ ]:
Validate = Model1.forward_propagation(train_val_images.T)
out_class=(np.argmax(Validate,axis=0))
target_class_validate=(np.argmax(train_val_labels,axis=1))
acc2 = 100*np.sum(out_class==target_class_validate)/Validate.shape[1]
print(acc2)

68.65


In [ ]:
train_imag.shape

(50000, 784)